In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model

In [2]:
import azureml.core
print(azureml.core.VERSION)

1.17.0


In [3]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

demo-ent-ws
demo
westeurope
bcbf34a7-1936-4783-8840-8f324c37f354


In [4]:
from azureml.core.model import Model

model = Model(ws, 'aml-wrkshp-classif-empl-attrition')

print('Model name: ', model.name, '\n', 'Model description: ', model.description, '\n', 'Model version: ', model.version, sep='')

Model name: aml-wrkshp-classif-empl-attrition
Model description: Binary classification model for employees attrition
Model version: 3


In [23]:
# Download the model locally in order to test the score.py script
model.download(target_dir='.', exist_ok=True)

'classif-empl-attrition.pkl'

In [16]:
%%writefile score.py

import os
import json
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
#from sklearn.externals import joblib
import joblib

from inference_schema.schema_decorators import input_schema, output_schema
from inference_schema.parameter_types.numpy_parameter_type import NumpyParameterType
from inference_schema.parameter_types.pandas_parameter_type import PandasParameterType


input_sample = pd.DataFrame(data=[{'Age': 49, 'BusinessTravel': 'Travel_Rarely', 'DailyRate': 1098, 'Department': 'Research & Development', 'DistanceFromHome': 4, 'Education': 2, 'EducationField': 'Medical', 'EnvironmentSatisfaction': 4, 'Gender': 'Female', 'HourlyRate': 21, 'JobInvolvement': 3, 'JobLevel': 2, 'JobRole': 'Laboratory Technician', 'JobSatisfaction': 3, 'MaritalStatus': 'Single', 'MonthlyIncome': 711, 'MonthlyRate': 2124, 'NumCompaniesWorked': 8, 'OverTime': 1, 'PercentSalaryHike': 8, 'PerformanceRating': 4, 'RelationshipSatisfaction': 3, 'StockOptionLevel': 0, 'TotalWorkingYears': 2, 'TrainingTimesLastYear': 0, 'WorkLifeBalance': 3, 'YearsAtCompany': 2, 'YearsInCurrentRole': 1, 'YearsSinceLastPromotion': 0, 'YearsWithCurrManager': 1}])
output_sample = np.array([0])


def init():
    # AZUREML_MODEL_DIR is an environment variable created during deployment. Join this path with the filename of the model file.
    # It holds the path to the directory that contains the deployed model (./azureml-models/$MODEL_NAME/$VERSION).
    # If there are multiple models, this value is the path to the directory containing all deployed models (./azureml-models).
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'classif-empl-attrition.pkl')
    
    # Deserialize the model file back into a sklearn model
    model = joblib.load(model_path)


@input_schema('data', PandasParameterType(input_sample))
@output_schema(NumpyParameterType(output_sample))
def run(data):
    try:
        result = model.predict(data)
        return json.dumps({"result": result.tolist()})
    except Exception as e:
        result = str(e)
        return json.dumps({"error": result})

Overwriting score.py


In [17]:
import sklearn
from azureml.core.environment import Environment

environment = Environment("LocalDeploy")

environment.python.conda_dependencies.add_conda_package("python=3.6.2")
environment.python.conda_dependencies.add_conda_package("pandas")
environment.python.conda_dependencies.add_conda_package("numpy")

environment.python.conda_dependencies.add_pip_package("azureml-core==1.17.0")
environment.python.conda_dependencies.add_pip_package("azureml-defaults==1.17.0")
environment.python.conda_dependencies.add_pip_package("sklearn-pandas")
environment.python.conda_dependencies.add_pip_package("inference-schema[numpy-support]")
environment.python.conda_dependencies.add_pip_package("joblib")
environment.python.conda_dependencies.add_pip_package("scikit-learn==0.22.2.post1")

In [18]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=environment)

In [19]:
from azureml.core.webservice import LocalWebservice

webservice_name = 'predict-attrition-test'

# This is optional, if not provided Docker will choose a random unused port.
deployment_config = LocalWebservice.deploy_configuration(port=6789)

local_service = Model.deploy(ws, webservice_name, [model], inference_config, deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry demoentwsb5fa738e.azurecr.io
Logging into Docker registry demoentwsb5fa738e.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM demoentwsb5fa738e.azurecr.io/azureml/azureml_1a20c7e2547432c1a93b79d84147bb47
 ---> 7b22859be9d7
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> 7b000f2b3f7f
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImJjYmYzNGE3LTE5MzYtNDc4My04ODQwLThmMzI0YzM3ZjM1NCIsInJlc291cmNlR3JvdXBOYW1lIjoiZGVtbyIsImFjY291bnROYW1lIjoiZGVtby1lbnQtd3MiLCJ3b3Jrc3BhY2VJZCI6IjU5NTcyYTM0LThiNmItNDM0ZS04ODMwLTI0NDlhMzEzMjI0MCJ9LCJtb2RlbHMiOnt9LCJtb2RlbHNJbmZvIjp7fX0= | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 7ecc414515ac
 ---> d90a3f4e3f2a
Step 4/5 : RUN mv '/var/azureml-app/tmpkx25s20o.py' /var/azureml-app/main.py
 ---> Running in 3aa5149978cd
 ---> 1035213703ea
Step 5/5 : CMD ["runsvdir","/var/run

In [20]:
local_service.wait_for_deployment()

Checking container health...
Local webservice is running at http://localhost:6789


In [21]:
print('Local service port: {}'.format(local_service.state))
print('Local service port: {}'.format(local_service.port))

Local service port: running
Local service port: 6789


In [22]:
print(local_service.get_logs())

2020-11-12T10:20:55,611419780+00:00 - rsyslog/run 
2020-11-12T10:20:55,613485559+00:00 - iot-server/run 
2020-11-12T10:20:55,615089598+00:00 - gunicorn/run 
2020-11-12T10:20:55,623347515+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_b417035e27d8ada91933498a4a62fd54/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b417035e27d8ada91933498a4a62fd54/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b417035e27d8ada91933498a4a62fd54/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b417035e27d8ada91933498a4a62fd54/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_b417035e27d8ada91933498a4a62fd54/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
EdgeHubC